In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 100)

In [95]:
zri_df=pd.read_csv('../data/Zip_Zri_AllHomesPlusMultifamily.csv')
zri_df=zri_df.reset_index()
zri_df_tmp=zri_df.drop(['RegionID','RegionName','City','State','Metro','CountyName','SizeRank'],axis=1)
zri_df_pivot=pd.DataFrame(zri_df_tmp.stack())
zri_df_pivot=zri_df_pivot[1:].reset_index()
zri_df_pivot.columns=['index','Date','Value']
zri_reshaped=zri_df_pivot.merge(zri_df[['index','RegionID','RegionName','City','State','Metro','CountyName','SizeRank']],on='index')
zri_reshaped=zri_reshaped.drop('index',axis=1)
zipcodes = zri_df['RegionName'].astype(str).unique()

zri_df['RegionName'] = zri_df['RegionName'].astype(str)
zri_df['RegionName'] = zri_df['RegionName'].map(lambda x: '0' + x if len(x) < 5 else x)
zipcodes = zri_df['RegionName'].unique()

In [4]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [72]:
# taxes_by_zip_df_og_11 = pd.read_csv('../../data/Taxes/2011zipcode/11zpallnoagi.csv',
#                                  index_col=None, dtype={'ZIPCODE': str})
# taxes_by_zip_df = taxes_by_zip_df_og_11[['STATE', 'ZIPCODE', 'N18425', 'A18300', 'A18425', 'A06500']]
# taxes_by_zip_df = taxes_by_zip_df.loc[taxes_by_zip_df['ZIPCODE'].isin(zipcodes)]
# taxes_by_zip_df['year'] = '2011'

In [77]:
# taxes_by_zip_df_og_10 = pd.read_csv('../../data/Taxes/2010zipcode/10zpallnoagi.csv',
#                                  index_col=None, dtype={'ZIPCODE': str})
# taxes_by_zip_df_10 = taxes_by_zip_df_og_10[['STATE', 'ZIPCODE', 'N18425', 'A18300', 'A18425', 'A06500']]
# taxes_by_zip_df_10 = taxes_by_zip_df_10.loc[taxes_by_zip_df_10['ZIPCODE'].isin(zipcodes)]
# taxes_by_zip_df_10['year'] = '2010'

# taxes_by_zip_df_10['state_local_perc'] = taxes_by_zip_df_10['A18425']/ \
# (taxes_by_zip_df_10['A18425']+taxes_by_zip_df_10['A06500'])
# taxes_by_zip_df = pd.concat([taxes_by_zip_df, taxes_by_zip_df_10])

In [96]:
taxes_by_zip_df_og_12 = pd.read_csv('../../data/Taxes/2012zipcode/12zpallnoagi.csv',
                                 index_col=None, dtype={'ZIPCODE': str})

taxes_by_zip_df_12 = taxes_by_zip_df_og_12[['STATE', 'ZIPCODE', 'N18425', 'A18300', 'A18425', 'A06500']]
taxes_by_zip_df_12 = taxes_by_zip_df_12.loc[taxes_by_zip_df_12['ZIPCODE'].isin(zipcodes)]
taxes_by_zip_df_12['year'] = '2012'
taxes_by_zip_df = taxes_by_zip_df_12
# taxes_by_zip_df = pd.concat([taxes_by_zip_df, taxes_by_zip_df_12])
taxes_by_zip_df['state_local_perc'] = taxes_by_zip_df['A18425']/ \
(taxes_by_zip_df['A18425']+taxes_by_zip_df['A06500'])


In [115]:
sum(zipcodes == '01013')

1

In [125]:
taxes_by_zip_df_og_12[taxes_by_zip_df_og_12['ZIPCODE'] == '01013']

,STATEFIPS,STATE,ZIPCODE,AGI_STUB,N1,MARS1,MARS2,MARS4,PREP,N2,...,N09600,A09600,N06500,A06500,N10300,A10300,N11901,A11901,N11902,A11902
10731,25,MA,01013,0,10090.0,5290.0,2850.0,1780.0,5440.0,18120.0,...,30.0,132.0,6710.0,31596.0,7070.0,33004.0,1090.0,2322.0,8710.0,21384.0


In [97]:
for i in range(13, 19, 1):
    temp_taxes = pd.read_csv(f'../../data/Taxes/zipcode20{i}/{i}zpallnoagi.csv',
                                 index_col=None, dtype={'ZIPCODE': str})
    temp_taxes = temp_taxes[['STATE', 'ZIPCODE', 'N18425', 'A18300', 'A18425', 'A06500']]
    temp_taxes = temp_taxes.loc[temp_taxes['ZIPCODE'].isin(zipcodes)]
    temp_taxes['state_local_perc'] = temp_taxes['A18425']/ \
        (temp_taxes['A18425']+temp_taxes['A06500'])
    temp_taxes["year"] = (f'20{i}')
    taxes_by_zip_df = pd.concat([taxes_by_zip_df, temp_taxes])

In [106]:
df = taxes_by_zip_df.sort_values(by=['ZIPCODE', 'year']).reset_index(drop=True) 
df = df[['ZIPCODE', 'year', 'state_local_perc']]
df

,ZIPCODE,year,state_local_perc
0,01001,2012,0.221197
1,01001,2013,0.213751
2,01001,2014,0.208784
3,01001,2015,0.206719
4,01001,2017,0.204536
...,...,...,...
90473,99901,2014,0.006957
90474,99901,2015,0.005519
90475,99901,2016,0.005549
90476,99901,2017,0.011413


In [107]:
# add 11 copies of every row
df= df.loc[df.index.repeat(12)].reset_index(drop=True) 

# add month column filled with 0
df['month'] = 0 

# get num unique zip codes
zips= df['ZIPCODE'].unique() 

# get num unique years
years = df['year'].unique() 

# fill in the month column
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] * int(df.shape[0]/12)
df['month'] = months

# replace all elements in rows with month != 12 with nan, 
# you will have to adjust the columns that you want to avoid replacing with nan
df.loc[df['month']!=12, df.columns[2]] = np.nan 
for item in zips:
    temp = df.loc[df['ZIPCODE']==item,:]
    temp['state_local_perc'] = temp['state_local_perc'].interpolate(method='linear')
#     for col in temp.columns[7]:
#         temp[col] = temp[col].interpolate(method='linear')
    df.loc[df['ZIPCODE']==item,:] = temp 

<ipython-input-107-d45dcceb1c92>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['state_local_perc'] = temp['state_local_perc'].interpolate(method='linear')


In [108]:
df

,ZIPCODE,year,state_local_perc,month
0,01001,2012,NaN,1
1,01001,2012,NaN,2
2,01001,2012,NaN,3
3,01001,2012,NaN,4
4,01001,2012,NaN,5
...,...,...,...,...
1085731,99901,2018,0.008740,8
1085732,99901,2018,0.008406,9
1085733,99901,2018,0.008072,10
1085734,99901,2018,0.007738,11


In [110]:
df.to_csv('../data/final_TAXES_by_zip.csv', index=False)

In [126]:
df[df['ZIPCODE'] == '01013']

,ZIPCODE,year,state_local_perc,month
360,01013,2012,NaN,1
361,01013,2012,NaN,2
362,01013,2012,NaN,3
363,01013,2012,NaN,4
364,01013,2012,NaN,5
365,01013,2012,NaN,6
366,01013,2012,NaN,7
367,01013,2012,NaN,8
368,01013,2012,NaN,9
369,01013,2012,NaN,10
